In [1]:
from python_speech_features import mfcc

In [2]:
import scipy.io.wavfile as wav

In [3]:
import numpy as np

In [4]:
from tempfile import TemporaryFile

In [5]:
import os

In [6]:
import pickle

In [7]:
import random

In [8]:
import operator

In [9]:
#function to find the distance between feature vectors and find the neighbours
def findNeighbour(trainSet , instance , k):
    #gets the distance between the instance and all the points in the trainset
    dist = []
    for x in range (len(trainSet)):
        tempDist = distance(trainSet[x] , instance , k) + distance(instance , trainSet[x] , k)
        dist.append((trainSet[x][2] , tempDist))
    
    #then we sort distances
    dist.sort(key = operator.itemgetter(1))
    
    #now we get the nearest k neighbours
    neighbours = []
    for x in range(k):
        neighbours.append(dist[x][0])
    return neighbours

In [10]:
#identify the class of the neighbours
def neighbourClass(neighbours):
    classVote = {}
    
    for x in range(len(neighbours)):
        response = neighbours[x]
        if response in classVote:
            classVote[response] += 1
        else:
            classVote[response] = 1
        
    sorter = sorted(classVote.items() , key = operator.itemgetter(1) , reverse = True)
    
    return sorter[0][0]

In [11]:
#to evaluate the model
def accuracy(testSet , prediction):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
            
    return (1.0 * correct) / len(testSet)

In [12]:
directory = "./music_data_set/Data/genres_original/"

In [13]:
f = open("my.dat" , "wb")

i = 0;

for folder in os.listdir(directory):
    i += 1
    if i == 11:
        break
    for file in os.listdir(directory+folder):        
        try:
            (rate, sig) = wav.read(directory+folder+"/"+file)
            mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, f)
        except Exception as e:
            print('Got an exception: ', e, ' in folder: ', folder, ' filename: ', file)        

f.close()

In [14]:
# Split the dataset into training and testing sets respectively
dataset = []

def loadData(filename, split, trainSet, testSet):
    with open('my.dat', 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
    for x in range(len(dataset)):
        if random.random() < split:
            trainSet.append(dataset[x])
        else:
            testSet.append(dataset[x])
trainingSet = []
testingSet = []
loadData('my.dat', 0.66, trainingSet, testingSet)

In [15]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance

In [31]:
# making predictions using KNN
leng = len(testingSet)
predictions = []
for x in range(leng):
    predictions.append(neighbourClass(findNeighbour(trainingSet, testingSet[x], 5)))

accuracy1 = accuracy(testingSet, predictions)
print(accuracy1*100)

67.31301939058172


In [25]:
testY = []
for x in range(len(testingSet)):
    testY.append(testingSet[x][-1])
testY

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,


In [20]:
predictions

[1,
 1,
 1,
 1,
 6,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 9,
 1,
 3,
 1,
 9,
 9,
 1,
 1,
 6,
 2,
 2,
 2,
 8,
 6,
 2,
 2,
 6,
 9,
 8,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 8,
 2,
 6,
 9,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 9,
 3,
 8,
 6,
 3,
 6,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 1,
 3,
 3,
 9,
 4,
 3,
 9,
 4,
 9,
 4,
 4,
 4,
 4,
 3,
 4,
 9,
 4,
 3,
 9,
 1,
 4,
 4,
 4,
 8,
 1,
 9,
 2,
 8,
 9,
 9,
 1,
 4,
 6,
 4,
 8,
 9,
 9,
 4,
 9,
 6,
 4,
 9,
 9,
 2,
 9,
 5,
 5,
 5,
 1,
 8,
 5,
 1,
 5,
 5,
 8,
 5,
 5,
 5,
 5,
 4,
 5,
 8,
 5,
 5,
 9,
 5,
 5,
 5,
 5,
 4,
 5,
 10,
 2,
 5,
 6,
 6,
 3,
 4,
 6,
 6,
 9,
 1,
 1,
 6,
 6,
 1,
 6,
 6,
 6,
 9,
 6,
 6,
 1,
 6,
 6,
 6,
 1,
 6,
 3,
 6,
 6,
 6,
 1,
 6,
 6,
 9,
 6,
 1,
 1,
 4,
 1,
 1,
 6,
 4,
 7,
 4,
 7,
 4,
 7,
 7,
 7,
 7,
 7,
 7,
 1,
 7,
 7,
 9,
 7,
 7,
 9,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 1,
 7,
 1,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,

In [30]:
from sklearn.metrics import accuracy_score
print(accuracy_score(testY, predictions)*100)

67.31301939058172


In [29]:
from sklearn.metrics import classification_report
print(classification_report(testY, predictions))

              precision    recall  f1-score   support

           1       0.46      0.72      0.56        29
           2       0.87      0.74      0.80        35
           3       0.71      0.84      0.77        38
           4       0.45      0.37      0.41        41
           5       0.86      0.63      0.73        30
           6       0.58      0.56      0.57        39
           7       1.00      0.79      0.88        38
           8       0.67      0.65      0.66        37
           9       0.48      0.60      0.53        40
          10       0.97      0.88      0.92        34

    accuracy                           0.67       361
   macro avg       0.70      0.68      0.68       361
weighted avg       0.70      0.67      0.68       361



In [ ]:
# testing the code with external samples

test_dir = "./music_data_set/Test/"

#hiphop
# test_file = test_dir + "test6.wav"

#blues
# test_file = test_dir + "test4.wav"

#pop
# test_file = test_dir + "test8.wav"

#rock
# test_file = test_dir + "test10.wav"

#jazz
# test_file = test_dir + "test7.wav"

#reggae
# test_file = test_dir + "test9.wav"

#metal
# test_file = test_dir + "test2.wav"

#country
# test_file = test_dir + "test.wav"

#disco
# test_file = test_dir + "test5.wav"

#classical
# test_file = test_dir + "test3.wav"

test_file = test_dir + "unknown11.wav"


In [ ]:
(rate, sig) = wav.read(test_file)
mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
covariance = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature = (mean_matrix, covariance, i)

In [ ]:
from collections import defaultdict
results = defaultdict(int)

results[1] = "HipHop"
results[2] = "Blues"
results[3] = "Pop"
results[4] = "Rock"
results[5] = "Jazz"
results[6] = "Reggae"
results[7] = "Metal"
results[8] = "Country"
results[9] = "Disco"
results[10] = "Classical"

results

In [ ]:
pred = neighbourClass(findNeighbour(dataset, feature, 4))
print(results[pred])